# Test

In [4]:
import requests
page = requests.get("https://stockbit.com/#/symbol/ICBP")
page

<Response [200]>

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [6]:
# scroll test
driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")

In [7]:
from selenium.webdriver.common.by import By

driver.get("https://stockbit.com/symbol/ICBP")

comment_elements = driver.find_elements(by=By.CLASS_NAME, value="stream-text")

comments = list()

for index, element in enumerate(comment_elements):
    print(index, element.text, "\n")
    comments.append(element.text)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.124)
Stacktrace:
0   chromedriver                        0x0000000102cefb7c chromedriver + 4209532
1   chromedriver                        0x0000000102c7b418 chromedriver + 3732504
2   chromedriver                        0x000000010292e368 chromedriver + 271208
3   chromedriver                        0x00000001029096fc chromedriver + 120572
4   chromedriver                        0x000000010298e660 chromedriver + 665184
5   chromedriver                        0x00000001029a0fbc chromedriver + 741308
6   chromedriver                        0x000000010295c59c chromedriver + 460188
7   chromedriver                        0x000000010295d644 chromedriver + 464452
8   chromedriver                        0x0000000102cc130c chromedriver + 4018956
9   chromedriver                        0x0000000102cc4fd8 chromedriver + 4034520
10  chromedriver  

In [ ]:
date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")

date = list()

for index, element in enumerate(date_elements):
    print(index, element.text, "\n")
    date.append(element.text)

In [ ]:
for a, b in zip([1,2], [3,4]):
    print(a,b)

In [ ]:
from dateutil import parser
parser.parse("1 Oct 22, 15:00") >= parser.parse("1 Oct 22")

In [ ]:
parser.parse("4 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29"[:16])

In [ ]:
date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")

while True:
    # Handle: ParserError: Unknown string format: 14 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29
    if parser.parse(date_elements[-2].text[:16]) >= parser.parse("1 Oct 22"):
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
        date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
    else:
        break
    

In [ ]:
# No guarantee that Selenium will stop in 1 Oct 22.
date_elements[-2].text

In [ ]:
from random import randint

randint(5,10)

In [ ]:
symbols = ["ICBP", "CPIN", "INDF", "MYOR", "CMRY", "GOOD", "MLBI"]

for symbol in symbols:
    print(f"https://stockbit.com/symbol/{symbol}")

In [ ]:
import pandas as pd

df = pd.DataFrame()
df['date'] = ['30 Sep 22, 09:22', '30 Sep 22, 09:23']
df['comment'] = ['$ICBP', '$ICBP2']
df['symbol'] = 'ICBP'

df_c = pd.DataFrame()

df_c['date'] = ['30 Sep 22, 09:22']
df_c['comment'] = ['$ICBP']
df_c['symbol'] = 'ICBP'

pd.concat([df, df_c])

In [ ]:
for index, (a, b) in enumerate(zip([1,2], [3,4])):
    print(index, a, b)

# Scraping

In [ ]:
symbols = ["ICBP", "CPIN", "INDF", "MYOR", "CMRY", "GOOD", "MLBI"]

from time import sleep

df = pd.DataFrame()

for symbol in symbols:
    driver.get(f"https://stockbit.com/symbol/{symbol}")
    
    while True:
        date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        
        # Handle: IndexError: list index out of range
        if len(date_elements) != 0:
            break

    while True:
        # date_elements[-2] Handle: StaleElementReferenceException or ''
        # .text[:16] Handle: ParserError: Unknown string format: 14 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29
        if parser.parse(date_elements[-2].text[:16]) >= parser.parse("1 Oct 22"):
            driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
            date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        else:
            break
    
    # value='stream-text-single' Handle: prevent Repost to be included in the comment_elements    
    comment_elements = driver.find_elements(by=By.CLASS_NAME, value="stream-text-single")
    
    dates = list()
    comments = list()
    
    for index, (date_element, comment_element) in enumerate(zip(date_elements, comment_elements)):
        # print(date_element.text, comment_element.text, "\n")
        
        # Handle: StaleElementReferenceException or ''
        if index == len(date_elements) - 1:
            break
        
        dates.append(date_element.text)
        comments.append(comment_element.text)
        
    print(f'symbol {symbol} '
          f'date_elements {len(date_elements)} dates {len(dates)} '
          f'comment_elements {len(comment_elements)} comments {len(comments)} ')
    
    df_c = pd.DataFrame()
    df_c['date'] = dates
    df_c['comment'] = comments
    df_c['symbol'] = symbol
    
    df = pd.concat([df, df_c])
    
    sleep(randint(5,10))

In [ ]:
# <div class="stream-text-single">
#   <a class="ext-link-indicator postid-9756129" data-entity-id="9756129" href="http://bit.ly/3bLj4Oc">http://bit.ly/3bLj4Oc</a>
# </div class="stream-text-single">
# the <a class..> won't be included.
df.iloc[444]['comment']

In [ ]:
df.to_csv("ICBP_MLBI.csv", index=False)

# Test

In [ ]:
import pandas as pd
from dateutil import parser

df = pd.read_csv("./ICBP_MLBI.csv")

# df['date'] = df['date'].apply(lambda row: parser.parse(row[:16]))
df.head()

In [ ]:
def get_comment(index):
    print(df['symbol'][index])
    print(df['comment'][index])

In [ ]:
def set_label(index: int, label_1: int, label_2: int):
    df.loc[index, 'label 1'] = label_1
    df.loc[index, 'label 2'] = label_2

In [ ]:
# get index after ICBP
print(df['symbol'].ne('ICBP').idxmax())
print(df['symbol'][460])

In [ ]:
import numpy as np
df['label 1'] = np.nan
df['label 2'] = np.nan
# index 0, sentiment neutral, type not spam
set_label(0, 0, 0)
# $ICBP
set_label(1, 0, 0)
# $GOTO main di 90 an aja ya kamu g usah kemana2, biar $ASII $TLKM $ADRO $ICBP dan bluechip yg lain bisa ngasih windress.
set_label(2, 0, 0)
# Info Singkat saham untuk 15 Desember 2022 (bukan rekomendasi jual-beli) :
set_label(3, 0, 0)
# @godbless @handusene sejak periode lockup open YU memang paling kenceng buang barang. Tapi hari ini YU gak manggung, CC jadi pemain utama.
set_label(4, 1, 0)
# $ULTJ fix tambah nyungsep liat chart kaya gini 🤯🤯
set_label(5, 1, 0)
# $HOKI yang merevolusi untuk menjadi $SRTG dikemudian hari, rasanya.
set_label(6, 2, 0)
# Risiko saham
set_label(7, 1, 0)
# Ko saya rada skeptis ya gaiss dengan kenaikan $GOTO hari ini. i mean, anjerr kalo beban yauda beban aja gt loo gausa ngesok naik segala terus besok2annya malah turun terus lagi..
set_label(8, 1, 0)
# $ICBP $INDF tes ombak ah...
set_label(9, 1, 0)
# Mirae Asset Sekuritas Indonesia Embun Pagi (13 Desember 2022)
set_label(10, 0, 0)
set_label(11, 2, 0)
set_label(12, 2, 0)
set_label(13, 2, 0)
set_label(14, 0, 0)
# IHSG Hari Ini Sulit Bangkit, Cek Saham BBRI, ICBP, hingga AKRA
set_label(15, 1, 0)
set_label(16, 2, 0)
set_label(17, 2, 0)
# Harga Kontainer Back To Normal
set_label(18, 0, 0)
# Join Zoom Meeting
set_label(19, 0, 1)
# Why KMDS ?
set_label(20, 2, 0)
set_label(21, 0, 0)
# @JS999 gw juga mau nuntut $ICBP mie soto gw g ada trauma makan mie soto,sekarang lebih milih makan mie goreng rendang
set_label(22, 0, 0)
# @JS999 makan Indomie lah ; biar $ICBP naik ..:)
set_label(23, 2, 0)
# Iya itulah, bukan hanya di Indonesia Pak. hampir disetiap negara kalau boleh transportasi itu gratis.
set_label(24, 0, 0)
set_label(25, 0, 0)
# Tertekan Akibat Harga Bahan Baku, Potensi Cuan Saham Indofood CBP (ICBP) Tetap Tinggi
set_label(26, 2, 0)
# Tertekan Akibat Harga Bahan Baku, Potensi Cuan Saham Indofood CBP (ICBP) Tetap Tinggi
set_label(27, 0, 0)
set_label(28, 0, 0)
set_label(29, 0, 0)
# $ICBP serius nanya parah suhu, udh lama ga lihat saham ini 3 bulanan bukan nya di 8 / 9rb.. sekarang koq naik ke 10rb ... ada apa gerangan ni , masih aman kah kalau masuk sekarang
set_label(30, 0, 0)
set_label(31, 0, 0)
set_label(32, 0, 0)
# ...
set_label(33, 0, 0)
set_label(34, 0, 0)
# $BBYB sesi 2 ARB kayanya
set_label(35, 1, 0)
# $ULTJ emang overhype IHSG turun kok malah naek sendiri
set_label(36, 1, 0)
# Stockpick : $ICBP Trading Buy TP 10.675
set_label(37, 2, 0)
# Kumpulan Artikel Pengaruh Jumlah Kepemilikan Investor Ritel terhadap Pergerakan Harga Saham
set_label(38, 0, 1)
# Pada tanggal 6 Desember 2022, $ICBP ditutup di titik tertingginya per hari ini di angka 10350, naik 0 poin (0,00%) dari harga closing kemarin. Secara teknikal dalam timeframe daily, harga ditutup di atas trend line di angka 10093,856. Sinyal BUY muncul sejak 7 November 2022, dan sinyal SPEC. SELL sempat muncul pada tanggal 18 November 2022 dan 22 November 2022.
set_label(39, 0, 0)
# Industry yg akan bertahan di akhir 2022 & Stockpick Q1 2023 yg mungkin udah banyak berseliweran apakah mungkin beli sekarang langsung naik ?
set_label(40, 0, 0)
# Mandiri Sekuritas: IHSG Sideways, Empat Saham Layak Dikoleksi Hari ini
set_label(41, 2, 0)
# izin belajar tentang $BOSS di @stockbit
#
#
#
#
# saya juga jualan produk indomie, kopi, gula dll. $ICBP $INDF $MYOR , boleh bapak/ibu berkenan beli dagangan saya. terima kasih
set_label(42, 0, 1)
set_label(43, 0, 0)
# @BimaAdiWiryo @RudiAlexander @Gungun08 @Tamami29 untuk saham $$INDF dan $ICBP prospeknya ud gak menarik karena selain labanya juga lagi turun, suka ada aksi2 yang merugikan pemegang saham seperti akuisisi pinehill yang bikin arus cashflow ke investor minoritas ngadat2 karena bayar hutang.. buat rekomendasi saham or buat yang mau nanya-nanya saham silahkan di follow ya
set_label(44, 1, 0)
# Skydrugz Corner: Emiten Dengan Goodwill Terbesar di Bursa
set_label(45, 1, 0)
set_label(46, 2, 0)
# @parsaulianpurba happy Cuan $ICBP
set_label(47, 2, 0)
# Analog switch off untuk jawa barat-tengah-timur sudah efektif awal desember ini.. another positive news buat emiten media kayak $SCMA ..
set_label(48, 2, 0)
set_label(49, 2, 0)
set_label(50, 0, 0)
set_label(51, 0, 0)
set_label(52, 0, 0)
set_label(53, 2, 0)
# kayaknya bentar lg ngoceh $INDF ma $ICBP ini abis makan indomie
set_label(54, 2, 0)
set_label(55, 1, 0)
# " Jangan membeli saham karna anda suka produknya, tapi beli lah saham karna anda paham bisnisnya maupun paham karakter dari ownernya "
set_label(56, 0, 0)
set_label(57, 2, 0)
set_label(58, 0, 0)
set_label(59, 0, 0)
# The essay for tech companies and tech adoption
set_label(60, 0, 1)
set_label(61, 0, 0)
set_label(62, 0, 0)
# Jenis Keunggulan Kompetitif
set_label(63, 0, 1)
set_label(64, 0, 0)
set_label(65, 0, 1)
set_label(66, 0, 1)
# Kendala modal dalam investasi
set_label(67, 0, 1)
set_label(68, 0, 0)
# @Elite saya tahun depan ingin fokus ke consumer goods, seperti $INDF, $ICBP $MYOR. karena mendekati tahun pemilu, biasanya suka ada bagi2 sembako/barang makanan lain. Diharapkan dapat meningkatkan sales.
set_label(69, 0, 0)
set_label(70, 0, 0)
set_label(71, 1, 0)
set_label(72, 2, 0)
set_label(73, 1, 0)
set_label(74, 0, 1)
set_label(75, 1, 0)
set_label(76, 1, 0)
set_label(77, 0, 0)
set_label(78, 1, 0)
set_label(79, 0, 0)
set_label(80, 0, 1)
set_label(81, 0, 0)
set_label(82, 1, 0)
set_label(83, 0, 0)
# $INDF $ICBP, lumayaaan...
# Revenue YOY Q3 naik 11%, QOQ naik 10,6%
# 
# Tapi net profitnya YOY turun (6,7 T vs 8 T) ?
# Iya karena kena beban keuangan sebesar 5,4 T dimana 3,1 T nya itu adalah RUGI NETO atas SELISIH NILAI TUKAR MATA UANG (lihat catatan kaki 31). Simplenya karena dollarnya sekarang ada di Rp. 15,727, kalau nanti dollar balik ke Rp 13,600 ya RUGI NETO 3,1 T ini semua hilang karena rugi kurs ini BELUM di REALIZED sampai jatuhnya tempo pembayaran obligasi INDF di 2031 dan 2051
# 
# Kalau rugi kurs ini kita abaikan, artinya net profit INDF harusnya 9,8 T... Dimana artinya naik sekitar 20% dibandingkan tahun lalu.. Yaa not bad lah...
set_label(84, 0, 0)
set_label(85, 0, 1)
set_label(86, 2, 0)
set_label(87, 0, 0)
set_label(88, 1, 0)
set_label(89, 2, 0)
set_label(90, 0, 0)
set_label(91, 0, 0)
set_label(92, 0, 1)
set_label(93, 0, 0)
set_label(94, 0, 1)
set_label(95, 2, 0)
set_label(96, 0, 1)
set_label(97, 0, 0)
set_label(98, 0, 0)
set_label(99, 0, 0)
# $ICBP selamat kepada top holding stockbit yang beli avg 10.000
set_label(100, 0, 0)
set_label(460, 2, 0)
set_label(461, 1, 0)
set_label(462, 0, 0)
set_label(463, 0, 0)
# NH Morning Brief – Dec. 14, 2022
set_label(464, 0, 1)
set_label(465, 0, 0)
# $JPFA $CPIN $CPRO $WMUU
# 
# Q4 biasanya revenue naik. Cocok di collect nih
set_label(466, 2, 0)
set_label(467, 0, 0)
set_label(468, 0, 0)
set_label(469, 2, 0)
set_label(470, 0, 1)
set_label(471, 0, 1)
set_label(472, 0, 1)
set_label(473, 1, 0)
set_label(474, 2, 0)
set_label(475, 1, 0)
set_label(476, 0, 0)
set_label(477, 0, 0)
# Nov_Des 2022 "seru" yah gocekan gerak harga saham. Nggak mau kalah dg gocekan Richarlison Brasil pada WorldCup kali ini... 😂
set_label(478, 0, 0)
set_label(479, 1, 0)
set_label(480, 0, 1)
set_label(481, 2, 0)
set_label(482, 1, 0)
# Raja Hutang semua itu kecuali $CPIN
set_label(483, 1, 0)
set_label(484, 0, 0)
# Skydrugz Corner: Emiten Pakan Ternak Undervalued?
# 
# Di postingan sebelumnya kita sudah bahas saham LQ45 https://cutt.ly/l1XlWpG
set_label(485, 0, 1)
set_label(486, 2, 0)
set_label(487, 2, 0)
# ayam-ayam 🐤
# 
# $JPFA kenapa lesu yah...
# 
# $MAIN $CPIN
# 
# memang semua sektor ada musimnya
set_label(488, 1, 0)
set_label(489, 0, 1)
set_label(490, 1, 0)
set_label(491, 1, 0)
set_label(492, 0, 0)
# $CPIN .. untuk ekspor bibit ayam sgt sulit, begitu ketat, byk uji laboratorium khusus nya mikrobiologi, dan saat ini cpin bs ekspor ke Singapura dan Timor Leste. emiten ini emang market leader nya di poultry, manajemen disini jd rujukan kompetitor nya, dan kebetulan saya pernah kerja disini. Oya liburan nataru bisa jd sentimen positif tuk poultry, selain inflasi yg akan melandai, harga komoditas yg mulai downtrend.
set_label(493, 2, 0)
set_label(494, 0, 0)
set_label(495, 0, 0)
set_label(496, 2, 0)
set_label(497, 1, 0)
# $CPIN break 5700 yuk
set_label(498, 2, 0)
set_label(499, 2, 0)
set_label(500, 0, 0)
set_label(501, 0, 0)
set_label(502, 0, 0)
# Have I ever told you
# I want you to the bone?
# Have I ever called you
# When you are all alone?
set_label(503, 0, 1)
set_label(504, 0, 0)
set_label(505, 0, 0)
# $WMUU melihat $CPIN dan $JPFA yg Q3 nya jatoh mending wns dlu tunggu konfirmasi Q3
set_label(506, 0, 0)
set_label(507, 0, 1)
set_label(508, 2, 0)
set_label(509, 0, 0)
# $WMUU jual emiten ini,

# ganti ke yang waras,

# $CPIN , $JPFA perusahaan bagus.
set_label(510, 1, 0)
set_label(511, 0, 0)
set_label(512, 2, 0)
set_label(513, 0, 0)
set_label(514, 0, 0)
set_label(515, 0, 0)
set_label(516, 0, 0)
set_label(517, 0, 0)
set_label(518, 0, 1)
set_label(519, 1, 0)
set_label(520, 0, 1)
set_label(521, 0, 1)
set_label(522, 0, 0)
set_label(523, 0, 0)
set_label(524, 2, 0)
set_label(525, 0, 0)
set_label(526, 1, 0)
set_label(527, 2, 0)
set_label(528, 0, 0)
set_label(529, 0, 1)
set_label(530, 0, 0)
set_label(531, 0, 0)
set_label(532, 1, 0)
set_label(533, 1, 0)
set_label(534, 0, 0)
set_label(535, 0, 0)
set_label(536, 0, 0)
set_label(537, 0, 0)
set_label(538, 0, 0)
set_label(539, 2, 0)
set_label(540, 0, 0)

In [ ]:
df_b = df.dropna().copy()
df_b['label 1'] = df_b['label 1'].astype(int)
df_b['label 2'] = df_b['label 2'].astype(int)

In [ ]:
df_b.to_csv("ICBP_MLBI_labeled.csv", index=False)

# Preprocessing

In [8]:
import pandas as pd
df_c = pd.read_csv("ICBP_MLBI_labeled.csv")
df_c.head()

,date,comment,symbol,label 1,label 2
0,"15 Dec 22, 16:48",POLL\nSusu paling enak apa gan? Vox Populi Vox...,ICBP,0,0
1,"15 Dec 22, 15:00",$ICBP,ICBP,0,0
2,"15 Dec 22, 10:25",$GOTO main di 90 an aja ya kamu g usah kemana2...,ICBP,0,0
3,"14 Dec 22, 19:45",Info Singkat saham untuk 15 Desember 2022 (buk...,ICBP,0,0
4,"14 Dec 22, 19:41",@godbless @handusene sejak periode lockup open...,ICBP,1,0


## Not Spam/Spam

- drop tagged user's name with `@[\w]+`
- drop tagged stock symbol `\$?[A-Z]{4}`
- drop tagged hashtag symbol `#[\w]+`
- drop url with `((https|http):\/\/|www.)[\w.\/]+`

In [9]:
label_2 = df_c.groupby(by='label 2').sample(2, random_state=1)['comment'].tolist()
label_2

['Lagi sedih2 nya saham ayam2an bang $JPFA $CPIN $WMUU $MAIN 😌',
 '$INDF\nSaya senang LK Indofood sudah keluar Q3. Saya perlu akui, hebat manajemen dalam arungi penjualan di 2022 ini. Saya sebagai pedagang Indomie, sih, tahun ini puji Tuhan, lancar. Orang masih ambil barang sama saya. Harga saya masih bisa lepas murah dibanding supermarket. Saya kagum dengan produknya, animo masyarakat masih suka mie nya.\n\n\nsukses terus pak Salim. Pengen kerja di perusahaan bapak, tapi ditolak HRD bapak, yang masih muda dan single itu.\n\nsaya akhirnya dagang indomie aja, supply ke org yg butuh.',
 "Kapan bisa FINANCIAL FREEDOM kayak Lo Kheng Hong?\n\nBapak LKH merupakan salah satu panutan saya dalam berinvestasi di pasar modal.\nSaya juga percaya untuk menang dengan cara lebih 'cepat' dan cerdas, kita harus melihat sejarah seseorang atau suatu negara.\n\nSaya membeli buku Bpk Lukas Setia Atmaja berjudul Lo Kheng Hong. Dimana buku ini secara lengkap menjelaskan perjalanan investasi Bapak LKH dan jug

## RegEx

In [10]:
import re

[re.sub('@[\w]+', '', item) for item in label_2]

['Lagi sedih2 nya saham ayam2an bang $JPFA $CPIN $WMUU $MAIN 😌',
 '$INDF\nSaya senang LK Indofood sudah keluar Q3. Saya perlu akui, hebat manajemen dalam arungi penjualan di 2022 ini. Saya sebagai pedagang Indomie, sih, tahun ini puji Tuhan, lancar. Orang masih ambil barang sama saya. Harga saya masih bisa lepas murah dibanding supermarket. Saya kagum dengan produknya, animo masyarakat masih suka mie nya.\n\n\nsukses terus pak Salim. Pengen kerja di perusahaan bapak, tapi ditolak HRD bapak, yang masih muda dan single itu.\n\nsaya akhirnya dagang indomie aja, supply ke org yg butuh.',
 "Kapan bisa FINANCIAL FREEDOM kayak Lo Kheng Hong?\n\nBapak LKH merupakan salah satu panutan saya dalam berinvestasi di pasar modal.\nSaya juga percaya untuk menang dengan cara lebih 'cepat' dan cerdas, kita harus melihat sejarah seseorang atau suatu negara.\n\nSaya membeli buku Bpk Lukas Setia Atmaja berjudul Lo Kheng Hong. Dimana buku ini secara lengkap menjelaskan perjalanan investasi Bapak LKH dan jug

In [11]:
[re.sub('\$?[A-Z]{4}', '', item) for item in label_2]

['Lagi sedih2 nya saham ayam2an bang     😌',
 '\nSaya senang LK Indofood sudah keluar Q3. Saya perlu akui, hebat manajemen dalam arungi penjualan di 2022 ini. Saya sebagai pedagang Indomie, sih, tahun ini puji Tuhan, lancar. Orang masih ambil barang sama saya. Harga saya masih bisa lepas murah dibanding supermarket. Saya kagum dengan produknya, animo masyarakat masih suka mie nya.\n\n\nsukses terus pak Salim. Pengen kerja di perusahaan bapak, tapi ditolak HRD bapak, yang masih muda dan single itu.\n\nsaya akhirnya dagang indomie aja, supply ke org yg butuh.',
 "Kapan bisa L DOM kayak Lo Kheng Hong?\n\nBapak LKH merupakan salah satu panutan saya dalam berinvestasi di pasar modal.\nSaya juga percaya untuk menang dengan cara lebih 'cepat' dan cerdas, kita harus melihat sejarah seseorang atau suatu negara.\n\nSaya membeli buku Bpk Lukas Setia Atmaja berjudul Lo Kheng Hong. Dimana buku ini secara lengkap menjelaskan perjalanan investasi Bapak LKH dan juga cara beliau analisa perusahaan.\n\n

In [12]:
[re.sub('#[\w]+', '', item) for item in label_2]

['Lagi sedih2 nya saham ayam2an bang $JPFA $CPIN $WMUU $MAIN 😌',
 '$INDF\nSaya senang LK Indofood sudah keluar Q3. Saya perlu akui, hebat manajemen dalam arungi penjualan di 2022 ini. Saya sebagai pedagang Indomie, sih, tahun ini puji Tuhan, lancar. Orang masih ambil barang sama saya. Harga saya masih bisa lepas murah dibanding supermarket. Saya kagum dengan produknya, animo masyarakat masih suka mie nya.\n\n\nsukses terus pak Salim. Pengen kerja di perusahaan bapak, tapi ditolak HRD bapak, yang masih muda dan single itu.\n\nsaya akhirnya dagang indomie aja, supply ke org yg butuh.',
 "Kapan bisa FINANCIAL FREEDOM kayak Lo Kheng Hong?\n\nBapak LKH merupakan salah satu panutan saya dalam berinvestasi di pasar modal.\nSaya juga percaya untuk menang dengan cara lebih 'cepat' dan cerdas, kita harus melihat sejarah seseorang atau suatu negara.\n\nSaya membeli buku Bpk Lukas Setia Atmaja berjudul Lo Kheng Hong. Dimana buku ini secara lengkap menjelaskan perjalanan investasi Bapak LKH dan jug

In [13]:
[re.sub('((https|http):\/\/|www.)[\w.\/]+', '', item) for item in label_2]

['Lagi sedih2 nya saham ayam2an bang $JPFA $CPIN $WMUU $MAIN 😌',
 '$INDF\nSaya senang LK Indofood sudah keluar Q3. Saya perlu akui, hebat manajemen dalam arungi penjualan di 2022 ini. Saya sebagai pedagang Indomie, sih, tahun ini puji Tuhan, lancar. Orang masih ambil barang sama saya. Harga saya masih bisa lepas murah dibanding supermarket. Saya kagum dengan produknya, animo masyarakat masih suka mie nya.\n\n\nsukses terus pak Salim. Pengen kerja di perusahaan bapak, tapi ditolak HRD bapak, yang masih muda dan single itu.\n\nsaya akhirnya dagang indomie aja, supply ke org yg butuh.',
 "Kapan bisa FINANCIAL FREEDOM kayak Lo Kheng Hong?\n\nBapak LKH merupakan salah satu panutan saya dalam berinvestasi di pasar modal.\nSaya juga percaya untuk menang dengan cara lebih 'cepat' dan cerdas, kita harus melihat sejarah seseorang atau suatu negara.\n\nSaya membeli buku Bpk Lukas Setia Atmaja berjudul Lo Kheng Hong. Dimana buku ini secara lengkap menjelaskan perjalanan investasi Bapak LKH dan jug

In [14]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/jason/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
s = '''Good muffins cost $3.88\nin New York.  Please buy me
... two of them.\n\nThanks.'''
print(word_tokenize(s))

['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


## Preprocessor

In [16]:
with open('./stopwords.txt', 'rt') as file_1:
    stopwords = [item.rstrip() for item in list(file_1)]

In [17]:
def preprocessor(text: str):
    # @KorneliusNelson 
    text = re.sub('@[\w]+', ' ', text)
    
    # BMTR
    # $MNCN
    text = re.sub('\$?[A-Z]{4}', ' ', text)
    
    # #investasi
    text = re.sub('#[\w]+', ' ', text)
    
    # https://cutt.ly/10dH3OY
    text = re.sub('((https|http):\/\/|www.)[\w.\/]+', ' ', text)
    
    # \n
    text = re.sub('\\n', ' ', text)
    
    # not word character
    text = re.sub('[^A-Za-z\s]', ' ', text)
    
    text = text.lower()
    
    text = text.strip()
    
    tokens = word_tokenize(text)
    
    text = ' '.join([word for word in tokens if word not in stopwords])
    
    return text

label_2_preprocessed = [preprocessor(item) for item in label_2]
label_2_preprocessed

['sedih nya saham ayam an bang',
 'senang lk indofood keluar q akui hebat manajemen arungi penjualan pedagang indomie sih tahun puji tuhan lancar orang ambil barang harga lepas murah dibanding supermarket kagum produknya animo masyarakat suka mie nya sukses pak salim pengen kerja perusahaan bapak ditolak hrd bapak muda single dagang indomie aja supply org yg butuh',
 'l dom kayak lo kheng hong bapak lkh satu panutan berinvestasi pasar modal percaya menang cara cepat cerdas melihat sejarah negara membeli buku bpk lukas setia atmaja berjudul lo kheng hong dimana buku lengkap menjelaskan perjalanan investasi bapak lkh cara beliau analisa perusahaan lkh investasi tahun investor pemula beragam gaya dicoba trading iya berebut saham ipo iya udah antri panjang eh cutloss hehehe lkh mendengar kisah warren buffett belajar value investing dimana ilmu praktikkan krisis moneter dimana membeli saham jutaan bertumbuh milyar milyar digunakan lkh membeli saham tahun merubah seluruh hidup lkh pas super 

# Neutral/Negative/Positive

In [18]:
label_1 = df_c.groupby(by='label 1').sample(2, random_state=1)['comment'].tolist()
label_1

['Phintraco Sekuritas: Pasar Wait and See Risalah FOMC The Fed, Perhatikan ADRO Hingga CPIN\nJAKARTA, Phintraco Sekuritas mengatakan pasar cenderung wait and see rilis risalah The Federal Open Market Committee (FOMC) The Fed. Ini akan mempengaruhi pergerakan IHSG sehingga diperkirakan cenderung terkoreksi ke kisaran 7.030-7.050 di Selasa (22/11/2022). IHSG hari ini diperkirakan akan bergera...',
 '$INTP\n\nB 9800-10000\nSL 9325 (5%)\nTP 11600 (18%)\n\nnfa dyor\n\ntag $BBNI $DOID $ICBP $MDKA',
 'Tak Memuaskan, Laba Indofood CBP (ICBP) Tergerus 33 Persen Sisa Rp3,30 Triliun\nEmitenNews.com—Emiten milik Grup Salim, PT Indofood CBP Sukses Makmur Tbk (ICBP) selama sembilan bulan pertama tahun 2022, mencatatkan kinerja kurang memuaskan. Hal ini berdasarkan penurunan laba bersih hingga 33,46 persen secara tahunan (YoY) menjadi Rp 3,30 triliun hingga kuartal III-2022, jauh ...',
 'Untuk menghindari inflasi serta kenaikan harga sembako menjelang natal dan tahun baru, ada baiknya Menteri Perdaga

In [19]:
label_1_preprocessed = [preprocessor(item) for item in label_1]
label_1_preprocessed

['phintraco sekuritas pasar wait and see risalah the fed perhatikan rta phintraco sekuritas pasar cenderung wait and see rilis risalah the federal open market committee the fed mempengaruhi pergerakan diperkirakan cenderung terkoreksi kisaran selasa hari diperkirakan bergera',
 'b sl tp nfa dyor tag',
 'memuaskan laba indofood cbp tergerus persen sisa rp triliun emitennews com emiten milik grup salim pt indofood cbp sukses makmur tbk sembilan bulan pertama tahun mencatatkan kinerja kurang memuaskan berdasarkan penurunan laba bersih persen tahunan yoy rp triliun kuartal iii jauh',
 'menghindari inflasi kenaikan harga sembako menjelang natal tahun baiknya menteri perdagangan kerjasama bandar bandar hebat markdown harga mengguyur emiten arb membantu menjaga kestabilan harga bahan pokok pasar',
 'diproyeksi menguat cermati saham rta indeks harga saham gabungan diperkirakan melanjutkan penguatan perdagangan selasa ditutup terkoreksi persen posisi perdagangan kemarin senin tim analis mnc sek

# Lemmatizer

In [20]:
with open('./slangwords.txt', 'rt') as file_1:
    slangwords = {}
    for line in file_1:
        (key, val) = line.rstrip().split(":")
        slangwords[key] = val
slangwords.get('&', 'default')

'dan'

In [21]:
slangwords['disclimer'] = 'disclaimer'
slangwords['nov'] = 'november'
slangwords['eksport'] = 'ekspor'
slangwords['lk'] = 'laporan keuangan'
slangwords['q'] = 'aku'
slangwords['bpk'] = 'bapak'
slangwords['ajah'] = 'saja'

stopwords_v2 = stopwords

stopwords_v2.append('disclaimer')
stopwords_v2.append('on')

stopwords_v2.append('phintraco')
stopwords_v2.append('sekuritas')

stopwords_v2.append('pasar')

stopwords_v2.append('open')
stopwords_v2.append('market')

stopwords_v2.append('ayam')

stopwords_v2.append('saham')

stopwords_v2.append('laporan keuangan')

stopwords_v2.append('sih')
stopwords_v2.append('an')
stopwords_v2.append('nya')

stopwords_v2.append('indomie')
stopwords_v2.append('indofood')
# stopwords_v2.append('salim')

# stopwords_v2.append('lkh')
# stopwords_v2.append('kheng')
# stopwords_v2.append('hong')

stopwords_v2.append('bapak')
stopwords_v2.append('buku')

stopwords_v2.append('ipo')

stopwords_v2 = list(stopwords)

# 2 words?
# stopwords_v2['disc on'] = 'disclaimer on'
# stopwords_v2['disclimer'] = 'disclaimer on'

# stopwords_v2.append('disclaimer on')

In [22]:
'disclaimer' in ['disclaimer on']

False

In [25]:
label_2[1]

'$INDF\nSaya senang LK Indofood sudah keluar Q3. Saya perlu akui, hebat manajemen dalam arungi penjualan di 2022 ini. Saya sebagai pedagang Indomie, sih, tahun ini puji Tuhan, lancar. Orang masih ambil barang sama saya. Harga saya masih bisa lepas murah dibanding supermarket. Saya kagum dengan produknya, animo masyarakat masih suka mie nya.\n\n\nsukses terus pak Salim. Pengen kerja di perusahaan bapak, tapi ditolak HRD bapak, yang masih muda dan single itu.\n\nsaya akhirnya dagang indomie aja, supply ke org yg butuh.'

In [24]:
[re.sub('\$[A-Z]{4}| [A-Z]{4} ', ' ', text) for text in ['Kapan bisa FINANCIAL FREEDOM kayak Lo Kheng Hong?', '$INDF ']]

['Kapan bisa FINANCIAL FREEDOM kayak Lo Kheng Hong?', '  ']

In [40]:
set({1})

{1}

In [ ]:
def preprocessor_v2(text: str):
    # @KorneliusNelson 
    text = re.sub('@[\w]+', ' ', text)
    
    # BMTR
    # $MNCN
    text = re.sub('\$[A-Z]{4}| [A-Z]{4} ', ' ', text)
    
    # # #investasi
    text = re.sub('#[\w]+', ' ', text)
    
    # # https://cutt.ly/10dH3OY
    text = re.sub('((https|http):\/\/|www.)[\w.\/]+', ' ', text)
    
    # # \n
    text = re.sub('\\n', ' ', text)
    
    # # not word character
    text = re.sub('[^A-Za-z\s]', ' ', text)
    
    text = text.lower()
    
    text = text.strip()
    
    tokens = word_tokenize(text)
    
    tokens = [slangwords.get(word, word) for word in tokens]
    
    tokens = [word for word in tokens if word not in stopwords_v2]
    
    text = ' '.join(tokens)
    
    return text

label_2_preprocessed_v2 = [preprocessor_v2(text) for text in label_2]
label_2_preprocessed_v2

['sedih bang',
 'senang keluar akui hebat manajemen arungi penjualan pedagang tahun puji tuhan lancar orang ambil barang harga lepas murah dibanding supermarket kagum produknya animo masyarakat suka mie sukses pak pengen kerja perusahaan ditolak hrd muda single dagang aja supply orang butuh',
 'financial freedom kayak satu panutan berinvestasi modal percaya menang cara cepat cerdas melihat sejarah negara membeli lukas setia atmaja berjudul dimana lengkap menjelaskan perjalanan investasi cara beliau analisa perusahaan investasi tahun investor pemula beragam gaya dicoba trading iya berebut iya udah antri panjang eh cutloss hehehe mendengar kisah warren buffett belajar value investing dimana ilmu praktikkan krisis moneter dimana membeli jutaan bertumbuh milyar milyar digunakan membeli tahun merubah seluruh hidup pas super pass alias kemana hehehe halaman mencatat daftar dimiliki tahun tahun jumlah pembelian persentase keuntungan menemukan rata rata pegangan pak tahunan persentase profit b

In [ ]:
label_1

['$JPFA akhir tahun 2021 tutup di 1720 posisi sekarang (14 nov 2022) 1370 koreksi 20%\n\n$CPIN akhir tahun 2021 tutup di 5950 posisi sekarang (14 nov 2022) 5800 koreksi 2.5%\n\n$MAIN akhir tahun 2021 tutup di 670 posisi sekarang (14 nov 2022) 550 koreksi 17.9%\n\n$JPFA masih ketinggalan dan layak di beli. apalagi dengan dibukanya eksport malaysia.\n\ndisc on.',
 '$CPIN Support berikutnya apabila tembus 5600 adalah 5525 dan 5425. Hindari dulu ini saham apabila tembus 5600 karena cenderung downtrend..',
 '$ICBP ZP ayo borong terus']

In [ ]:
label_1_preprocessed_v2 = [preprocessor_v2(text) for text in label_1]
label_1_preprocessed_v2

['akhir tahun tutup posisi november koreksi akhir tahun tutup posisi november koreksi akhir tahun tutup posisi november koreksi ketinggalan layak beli dibukanya ekspor malaysia disc on',
 'support berikutnya tembus hindari saham tembus cenderung downtrend',
 'zp ayo borong']

In [ ]:
with open('./slangwords_v2.txt', 'wt') as file_1:
    for word in slangwords:
        file_1.write(f'{word}:{slangwords.get(word)}\n')

In [ ]:
with open('./stopwords_v2.txt', 'wt') as file_1:
    for word in stopwords_v2:
        file_1.write(f'{word}\n')

Input: symbol, comment.

Misi dari Rules based adalah mencegah salah subjek.

1. lagi ngomong GOTO, tapi dia tag $INDF. -> komentar ada di $INDF.

**komentar ada di saham $INDF**
1. ['GOTO adalah saham gorengan \n $INDF'] dicocokin sama symbol -> ini dibuang

```
symbols = ['GOTO', 'INDF', 'ICBP']
symbols.delete(symbol)

# ini jangan dipakai, hanya pseudocode.
for word in comment.split():
  if word in symbols:
    # REJECT JANGAN DIMASUKIN ke NLP Sentiment.

['GOTO adalah saham gorengan'] -> ['GOTO', 'adalah', 'saham', 'gorengan']
```

2. ['$INDF adalah saham gorengan \n $GOTO'] -> ini dilanjutin ke NLP Sentiment.